In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt, freqz
from scipy.stats import pearsonr
import shutil
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import get_data_class, load, save

In [4]:
## propofolPuffTone
# session = 'Mary-Anesthesia-20160809-01'
session = 'MrJones-Anesthesia-20160105-01'

## propofolWakeup
# session = 'Mary-Anesthesia-20170203-02'

## Load Neural Data and Split Into Segments

In [8]:
def split_data_into_windows(session, all_data_dir, save_dir=None, windows=[2.5], strides=None, seconds_to_chop=30):
    if strides is None:
        strides = windows
    else:
        if len(strides) != len(windows):
            raise ValueError(f"If provided, the lists windows and strides must be the same length. Currently windows = {windows} and strides = {strides}")
    
    if save_dir is None:
        save_dir = all_data_dir
    
    data_class = get_data_class(all_data_dir, session)

    filename = os.path.join(all_data_dir, data_class, f'{session}.mat')
    print("Loading data ...")
    start = time.process_time()
    lfp, lfp_schema = loadmat(filename, variables=['lfp', 'lfpSchema'], verbose=False)
    T = lfp.shape[0]
    dt = lfp_schema['smpInterval'][0]

    print(f"Data loaded (took {time.process_time() - start:.2f} seconds)")
    
    end_step = T - int(seconds_to_chop/dt) # take off seconds_to_chop seconds from the end
    data = lfp[:end_step]
    for window, stride in zip(windows, strides):
        data_dir = os.path.join(save_dir, data_class, f"{session}_window_{window}_stride_{stride}")
        os.makedirs(data_dir, exist_ok=True)
        
        print(f"window = {window} s")
        num_windows = int(end_step/int(stride/dt))
        for i in tqdm(range(num_windows)):
            start = int(stride/dt)*i
            end = int(stride/dt)*i + int(window/dt)
            window_data = data[start:end]
            
            window_dict = {'data': window_data, 'start_ind': start, 'start_time': start*dt, 'end_ind': end, 'end_time': end*dt}
            save(window_dict, os.path.join(data_dir, f"window_{i}"))

In [6]:
all_data_dir = f"/om/user/eisenaj/datasets/anesthesia/mat"

In [7]:
split_data_into_windows(session, all_data_dir, all_data_dir)

Loading data ...
Data loaded (took 192.18 seconds)
window = 2.5 s


  0%|          | 0/4977 [00:00<?, ?it/s]

## Compile Results and Save

In [4]:
# folder_to_compile = "/om/user/eisenaj/ChaoticConsciousness/results/propofol/causality/causality_Mary-Anesthesia-20160809-01_window_2.5_stride_2.5"
# assert os.path.exists(folder_to_compile)

In [10]:
def compile_folder(folder_to_compile):
    files = os.listdir(folder_to_compile)

    all_window_data = []
    for file in tqdm(files):
        window_data = load(os.path.join(folder_to_compile, file))
        all_window_data.append(window_data) 

    results_df = pd.DataFrame(all_window_data)

    results_df = results_df.sort_values('start_ind')
    results_df = results_df.reset_index(drop=True)

    shutil.rmtree(folder_to_compile)

    save(results_df, folder_to_compile)

In [11]:
results_dir = "/om/user/eisenaj/ChaoticConsciousness/results/propofolPuffTone/VAR/VAR_Mary-Anesthesia-20160809-01_window_2.5_stride_2.5_Feb-03-2022_1546"
for folder in tqdm(os.listdir(results_dir)):
    compile_folder(os.path.join(results_dir, folder))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2822 [00:00<?, ?it/s]

  0%|          | 0/2822 [00:00<?, ?it/s]

  0%|          | 0/2822 [00:00<?, ?it/s]

  0%|          | 0/2822 [00:00<?, ?it/s]

  0%|          | 0/2822 [00:00<?, ?it/s]